In [65]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
train=pd.read_csv('preprocessed_train.csv')
test=pd.read_csv('/content/prepreocessed_test.csv')

In [ ]:
train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Marital_Status,Car_Ownership,Has_Children,ProdTaken
0,0,50.0,Self Enquiry,2,15.0,Large Business,male,1,4.0,basic,3,5.0,1,4,executive,253905.0,unmarried,0,0,1
1,1,56.0,Company Invited,1,14.0,Salaried,male,1,4.0,standard,3,2.0,1,4,senior manager,404475.0,divorced,1,0,0
2,2,NaN,Self Enquiry,1,10.0,Large Business,female,1,3.0,basic,3,4.0,0,4,executive,278145.0,married,0,0,1
3,3,20.0,Self Enquiry,2,18.0,Small Business,female,1,3.0,standard,4,1.0,0,5,senior manager,326805.0,divorced,1,0,0
4,4,48.0,Company Invited,3,17.0,Small Business,female,1,3.0,basic,4,4.0,0,4,executive,258435.0,single,1,0,1


# **최빈값으로 NULL 대체**

In [67]:
for column in train.columns:
    if train[column].isnull().any():  # 널값이 있는 경우에만
        mode_value = train[column].mode()[0]  # 최빈값 계산
        train[column].fillna(mode_value, inplace=True)  # 널값을 최빈값으로 채움
for column in test.columns:
    if test[column].isnull().any():  # 널값이 있는 경우에만
        mode_value = test[column].mode()[0]  # 최빈값 계산
        test[column].fillna(mode_value, inplace=True)  # 널값을 최빈값으로 채움

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   Age                     3489 non-null   float64
 2   TypeofContact           3489 non-null   object 
 3   CityTier                3489 non-null   int64  
 4   DurationOfPitch         3489 non-null   float64
 5   Occupation              3489 non-null   object 
 6   Gender                  3489 non-null   object 
 7   NumberOfPersonVisiting  3489 non-null   int64  
 8   NumberOfFollowups       3489 non-null   float64
 9   ProductPitched          3489 non-null   object 
 10  PreferredPropertyStar   3489 non-null   int64  
 11  NumberOfTrips           3489 non-null   float64
 12  Passport                3489 non-null   int64  
 13  PitchSatisfactionScore  3489 non-null   int64  
 14  Designation             3489 non-null   

# **원 핫 인코딩을 통한 처리**

In [68]:
train_gd=pd.get_dummies(train)
test_gd=pd.get_dummies(test)

In [69]:
prod_taken=train_gd.pop('ProdTaken')
train_gd['ProdTaken'] = prod_taken

In [ ]:
test_gd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             3489 non-null   int64  
 1   Age                            3489 non-null   float64
 2   CityTier                       3489 non-null   int64  
 3   DurationOfPitch                3489 non-null   int64  
 4   NumberOfPersonVisiting         3489 non-null   float64
 5   NumberOfFollowups              3489 non-null   float64
 6   PreferredPropertyStar          3489 non-null   float64
 7   NumberOfTrips                  3489 non-null   float64
 8   Passport                       3489 non-null   int64  
 9   PitchSatisfactionScore         3489 non-null   int64  
 10  MonthlyIncome                  3489 non-null   float64
 11  Car_Ownership                  3489 non-null   int64  
 12  Has_Children                   3489 non-null   i

# **모델 구성**

In [70]:
X_train = train_gd.iloc[:,0:33]
Y_train = train_gd.iloc[:,33]
X_test=test_gd.iloc[:,0:33]
X_train=X_train.astype('float32')
Y_train=Y_train.astype('float32')

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# 데이터 분할
X_train, X_val, Y_train, Y_val = train_test_split(
    train_gd.iloc[:, 0:33].astype('float32'),
    train_gd.iloc[:, 33].astype('float32'),
    test_size=0.25,  # 25%를 검증 데이터로 사용
    random_state=43  # 재현성을 위해 랜덤 시드 설정
)

# XGBoost 모델 생성
model = xgb.XGBClassifier(
    objective='binary:logistic',  # 이진 분류
    eval_metric=['logloss', 'error'],         # 평가 지표
    use_label_encoder=False,        # 경고 방지
    n_estimators=1000,              # 트리 개수
    max_depth=100,                   # 트리 깊이
    learning_rate=0.001             # 학습률
)

# 모델 학습 (검증 세트 포함)
model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], verbose=True)

# 테스트 세트에 대한 예측
Y_pred = model.predict_proba(X_test)

# 예측 결과 출력
print("테스트 세트에 대한 예측 결과:")
print(Y_pred)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:22:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.39757	validation_0-error:0.12715
[1]	validation_0-logloss:0.39740	validation_0-error:0.12715
[2]	validation_0-logloss:0.39724	validation_0-error:0.12715
[3]	validation_0-logloss:0.39707	validation_0-error:0.12715
[4]	validation_0-logloss:0.39690	validation_0-error:0.12715
[5]	validation_0-logloss:0.39673	validation_0-error:0.12715
[6]	validation_0-logloss:0.39657	validation_0-error:0.12715
[7]	validation_0-logloss:0.39640	validation_0-error:0.12715
[8]	validation_0-logloss:0.39623	validation_0-error:0.12715
[9]	validation_0-logloss:0.39607	validation_0-error:0.12715
[10]	validation_0-logloss:0.39590	validation_0-error:0.12715
[11]	validation_0-logloss:0.39573	validation_0-error:0.12715
[12]	validation_0-logloss:0.39556	validation_0-error:0.12715
[13]	validation_0-logloss:0.39541	validation_0-error:0.12715
[14]	validation_0-logloss:0.39524	validation_0-error:0.12715
[15]	validation_0-logloss:0.39507	validation_0-error:0.12715
[16]	validation_0-logloss:0.39491	

AttributeError: 'XGBClassifier' object has no attribute 'best_params_'

In [73]:
# ID 값을 가져오기 (test_gd의 첫 번째 열이 ID라고 가정)
test_ids = test_gd.iloc[:, 0]  # ID가 첫 번째 열에 있다고 가정

# 결과 데이터프레임 생성
results = pd.DataFrame({
    'ID': test_ids,
    'Prediction': Y_pred[:, 1]
})

# 값만 포함된 CSV 파일로 저장 (속성값 없이)
results.to_csv('submission.csv', index=False, header=False)

# **그리드 추가 모델 **

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 데이터 분할
X_train, X_val, Y_train, Y_val = train_test_split(
    train_gd.iloc[:, 0:33].astype('float32'),
    train_gd.iloc[:, 33].astype('float32'),
    test_size=0.25,  # 25%를 검증 데이터로 사용
    random_state=43  # 재현성을 위해 랜덤 시드 설정
)

# XGBoost 모델 생성
model = xgb.XGBClassifier(
    objective='binary:logistic',  # 이진 분류
    eval_metric='logloss',         # 평가 지표
    use_label_encoder=False,        # 경고 방지
)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 500, 1000],  # 트리 개수
    'max_depth': [3, 5, 7, 10],         # 트리 깊이
    'learning_rate': [0.01, 0.1, 0.2]   # 학습률
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='accuracy',  # 정확도를 기준으로 평가
                           cv=5,                # 5-겹 교차 검증
                           verbose=1,          # 진행 상황 출력
                           n_jobs=-1)          # 모든 CPU 코어 사용

# 모델 학습
grid_search.fit(X_train, Y_train)

# 최적의 하이퍼파라미터와 성능 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)
print("최고 정확도:", grid_search.best_score_)

# 최적의 모델로 검증 세트 평가
best_model = grid_search.best_estimator_
Y_pred = best_model.predict(X_val)

# 예측 결과 출력
print("검증 세트에 대한 정확도:", accuracy_score(Y_val, Y_pred))
print(classification_report(Y_val, Y_pred))


Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:19:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


최적의 하이퍼파라미터: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}
최고 정확도: 0.8742384656926424
검증 세트에 대한 정확도: 0.8888888888888888
              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94       762
         1.0       0.61      0.35      0.45       111

    accuracy                           0.89       873
   macro avg       0.76      0.66      0.69       873
weighted avg       0.87      0.89      0.88       873

